In [1]:
#Importing libraries and packages

import os 
import json

from keras.preprocessing.image import img_to_array, load_img
from keras.utils.data_utils import get_file
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model

import h5py
import numpy as np
import pickle as pk

Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Loading Models for comparison
first_check = VGG16(weights='imagenet') #is it a car?
second_check = pk.load(open("C:\\Users\\nickm\\PythonProjects\\FullStackDataScience\\car_damage_check\\classifier.pickle", 'rb')) #if yes, is it damaged?
third_check = pk.load(open("C:\\Users\\nickm\\PythonProjects\\FullStackDataScience\\car_damage_directional\\classifier.pickle", 'rb'))#if yes, where?
fourth_check = pk.load(open("C:\\Users\\nickm\\PythonProjects\\FullStackDataScience\\car_damage_severity\\classifier.pickle", 'rb'))#if yes, how badly?

W0714 15:32:37.110555 12532 deprecation_wrapper.py:119] From c:\users\nickm\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0714 15:32:37.123531 12532 deprecation_wrapper.py:119] From c:\users\nickm\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0714 15:32:37.125570 12532 deprecation_wrapper.py:119] From c:\users\nickm\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0714 15:32:37.144534 12532 deprecation_wrapper.py:119] From c:\users\nickm\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. P

In [4]:
#Loading valid categories for identifying cars using VGG16
with open('cat_counter.pk', 'rb') as f:
    cat_counter = pk.load(f)

In [5]:
cat_list = [k for k, v in cat_counter.most_common()[:27]]

In [17]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n02930766', 'cab'),
 ('n02974003', 'car_wheel'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04252225', 'snowplow'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03345487', 'fire_engine'),
 ('n03445924', 'golfcart'),
 ('n03791053', 'motor_scooter'),
 ('n03776460', 'mobile_home'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian'),
 ('n03417042', 'garbage_truck'),
 ('n02965783', 'car_mirror')]

# Car Check

In [6]:
CLASS_INDEX = None

In [7]:
CLASS_INDEX_PATH = "C:\\Users\\nickm\\PythonProjects\\FullStackDataScience\\imagenet_class_index.json"

In [8]:
def get_predictions(preds, top=5):
    global CLASS_INDEX
    
    #load json file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
        
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [9]:
def prepare_img_224(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [36]:
def car_categories_check(img_224, model):
    print ("Validating the picture of your car...")
    out = model.predict(img_224)
    top = get_predictions(out, top=5)
    for j in top[0]:
        if j[0:2] in cat_list:
            print("Car check passed. Proceeding to damage detector.")
            print("\n")
            return True
    return False
    

# Damage Check

In [37]:
def car_damage_check(classifier):
    print("Checking for damage...")
    base_model = first_check
    train_labels = ['Damaged', 'Not Damaged']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    
    if train_labels[preds[0]] == 'Damaged':
        print("Damage check complete - proceeding to damage location and severity detector.")
        print("\n")
        return True
    else:
        return False

# Damage Locator

In [38]:
def damage_locator(classifier):
    print("Locating area damaged - front, side or rear.")
    base_model = first_check
    train_labels = ['Front Damage', 'Side Damage', 'Rear Damage']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    
    print("Your car is damaged at - " + train_labels[preds[0]])
    print("Damage location assessment complete.")
    print("\n")

# Severity Assessment

In [39]:
def severity_assessment(classifier):
    print("Evaluating the severity of the damage.")
    base_model = first_check
    train_labels = ['Minor Damage', 'Moderate Damage', 'Severe Damage']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    
    print("The damage done to your car is rated as - " + train_labels[preds[0]])
    print("Damage severity estimate complete")
    print("\n")
    print("Thanks for testing my model!")

# Engine

In [40]:
def engine(img):
    while True:
        try:
            img_224 = prepare_img_224(img)
            g1 = car_categories_check(img_224, first_check)
            
            if g1 is False:
                print("Could not recognize picture of car. Please submit a different picture.")
                print("Hint: try a different angle, different lighting, or zooming in/out.")
                print("Picture must be in .jpg format.")
                break
                
            g2 = car_damage_check(second_check)
            
            if g2 is False:
                print("Could not recognize picture of car. Please submit a different picture.")
                print("Hint: try a different angle, different lighting, or zooming in/out.")
                print("Picture must be in .jpg format.")
                break
                
            x = damage_locator(third_check)
            y = severity_assessment(fourth_check)
            break
        except:
            print("Image not accessible. Please try again")
            break

In [41]:
img_path = "C:\\Users\\nickm\\Datasets\\car-damage-dataset\\car2.jpg"

In [42]:
engine(img_path)

Validating the picture of your car...
Car check passed. Proceeding to damage detector.


Checking for damage...
Damage check complete - proceeding to damage location and severity detector.


Locating area damaged - front, side or rear.
Your car is damaged at - Front Damage
Damage location assessment complete.


Evaluating the severity of the damage.
The damage done to your car is rated as - Moderate Damage
Damage severity estimate complete


Thanks for testing my model!
